# I.Import Necessary Dependencies and Settings¶

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import gensim
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
pd.options.display.max_colwidth = 200
%matplotlib inline

# II.Dataset Import

In [2]:
df = pd.read_csv('UnlistedTagalog1.csv', error_bad_lines=False)
pd.set_option('display.max_columns', None)
df.head()


,score,text
0,1,uy gagi wag kang magpaniwala haka hakang matagumpay green revolution panahong marcos ma lalong nalugmok kahirapan pilipino lumaganap malnutrisyon kabataan bumagsak kita mama...
1,2,marcos duterte pinag parehong tuta diktador pasista neveragain marcosnotahero dutertewakasan oustduterte
2,2,malinaw malinaw malinaw pinapatagal pandemya alam pamahalaan kayang pabagsakin mamamayan sama samang pagkilos sagot covid paghihirap sambayanan iisa oust duterte
3,1,marcos magiging bayani basahin buong pahayag nnara youth uplb ngayong martial law commemoration neveragain oustduterte
4,2,neveragaintomartiallaw resist rehabilitation marcos family stop duterte creating another political dynasty oust duterte


# III.Data Cleaning

In [3]:
# Convert to list
data = df.text.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
print(data[:1])

[' uy gagi wag kang magpaniwala haka hakang matagumpay green revolution panahong marcos ma lalong nalugmok kahirapan pilipino lumaganap malnutrisyon kabataan bumagsak kita mamamayan neveragain neverforget oustduterte ']


### Tokenize

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['uy', 'gagi', 'wag', 'kang', 'magpaniwala', 'haka', 'hakang', 'matagumpay', 'green', 'revolution', 'panahong', 'marcos', 'ma', 'lalong', 'nalugmok', 'kahirapan', 'pilipino', 'lumaganap', 'malnutrisyon', 'kabataan', 'bumagsak', 'kita', 'mamamayan', 'neveragain', 'neverforget', 'oustduterte']]


### Stemming

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [6]:
nlp = spacy.load("en_core_web_sm")
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

['lumaganap malnutrisyon neveragain neverforget oustduterte', 'neveragain oustduterte']


# IV.Create the Document-Word matrix

In [7]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  
# num chars > 3
                             max_features=50000)           
# max number of uniq words    
data_vectorized = vectorizer.fit_transform(data_lemmatized)

# V.Build LDA model with sklearn

In [8]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)


In [9]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.9,
                          learning_method='online', learning_offset=10.0, max_doc_update_iter=100, 
                          max_iter=10, mean_change_tol=0.001, n_components=10, n_jobs=-1, 
                          perp_tol=0.1,random_state=100, topic_word_prior=None, total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_decay=0.9, learning_method='online',
                          n_jobs=-1, random_state=100)

# VI.Diagnose model performance with perplexity and log-likelihood

In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
print(lda_model.get_params())

Log Likelihood:  -27205.80947678402
Perplexity:  157.38931922608336
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 10, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [11]:
# Define Search Param
search_params = {'n_components': [5,6,7,8,9,10], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)


GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10]})

In [12]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 5}
Best Log Likelihood Score:  -6946.331567205592
Model Perplexity:  145.32573375060855


# VII.Dominant topic

In [13]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.050000,0.050000,0.050000,0.480000,0.370000,3
Doc1,0.070000,0.070000,0.070000,0.730000,0.070000,3
Doc2,0.600000,0.100000,0.100000,0.100000,0.100000,0
Doc3,0.050000,0.050000,0.050000,0.800000,0.050000,3
Doc4,0.860000,0.030000,0.030000,0.030000,0.030000,0
Doc5,0.130000,0.020000,0.020000,0.180000,0.640000,4
Doc6,0.070000,0.070000,0.070000,0.730000,0.070000,3
Doc7,0.050000,0.050000,0.050000,0.480000,0.370000,3
Doc8,0.050000,0.050000,0.050000,0.800000,0.050000,3
Doc9,0.040000,0.040000,0.040000,0.600000,0.280000,3


In [14]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head(10)

,add,address,admin,administration,allege,allow,amp,anniversary,apologist,aquino,ask,attack,audit,bansa,bata,bayan,bbm,bbmforpresident,believe,bring,budget,campaign,candidate,case,change,class,come,continue,corruption,country,court,covid,cross,dami,davao,daw,day,deal,death,defend,dictator,dictatorship,dilawan,doh,drug,duqueresign,duterte,dutertemanyak,duterterapist,dutertewakasan,elcac,election,eleksyon,end,experience,face,fact,family,fight,filipino,forget,freedom,friend,fund,government,govt,group,hahahaha,happen,health,hear,help,history,hope,interview,investigation,issue,job,kayo,kill,know,lambaste,land,law,lawyere,leader,leave,leni,let,lie,life,live,lol,look,lose,make,marcose,mayor,mean,medium,member,mind,naman,nating,nation,need,neveragain,neverforget,news,night,niyo,number,office,opposition,order,oust,oustduterte,oustdutertenow,pala,pay,people,philippine,pilipina,police,policy,politic,politician,post,power,pre,presidency,president,press,probe,protect,puro,read,record,regime,register,remember,report,respond,response,result,return,run,say,senator,start,state,statement,stop,story,student,supply,support,supporter,talaga,talk,tell,term,test,testing,thank,thing,think,time,today,truth,try,twitter,tyranny,use,vaccine,vice,victim,vote,voter,wag,wake,want,war,watch,way,win,work,world,year
Topic0,0.646828,11.277504,4.204404,0.388808,0.664118,3.741393,1.911088,0.425281,16.340879,4.083617,28.853394,4.317945,1.262159,8.699711,4.305438,0.436375,1.910633,0.381189,0.392280,0.479664,0.428509,0.735248,0.558470,9.579450,1.304388,21.051631,6.410007,0.531771,0.417021,0.740383,0.491801,6.194039,11.036053,0.346955,0.482032,0.361375,0.860598,0.461025,0.855877,0.443710,2.478931,0.873655,0.670910,16.395090,0.370945,0.673123,407.590048,0.323528,0.374771,0.667101,0.434390,0.418342,5.857345,0.704699,0.494515,27.435742,3.119096,2.349295,2.693993,0.475722,0.437006,0.737069,3.632556,3.634872,1.114720,9.156439,2.039991,0.518926,2.961465,0.860556,1.010316,2.342392,0.743845,4.987678,0.354858,4.672712,3.764245,7.479763,57.770185,0.413161,0.469911,8.508044,0.349792,14.151382,25.021150,0.379451,1.029881,0.349244,0.435405,0.803096,0.760048,1.054409,0.505783,1.745376,3.259503,10.599195,0.627835,9.423143,0.408438,0.487468,0.346764,0.458069,12.602420,0.323793,0.789613,2.435691,0.746909,0.850058,4.885652,4.460542,3.408847,5.450758,0.792808,0.428786,1.355063,66.330460,2.010433,0.692252,0.417368,0.357742,46.493861,3.110688,17.040079,2.360227,1.945687,1.080947,0.806006,7.763780,0.671453,12.733619,0.646941,6.303862,1.233845,1.126053,0.556553,0.452391,9.080983,0.502054,2.459801,0.372406,3.106740,4.286733,3.968604,0.473339,9.929188,1.354205,0.920575,104.872792,0.608352,17.023052,2.422316,7.219136,4.697302,1.383340,3.500432,2.296431,0.470931,16.544157,0.408751,1.984185,1.313789,0.382499,17.814752,12.200728,0.631889,5.567147,26.865466,0.895201,0.498320,0.570454,0.435141,0.388561,0.422210,3.470867,0.369731,0.931403,0.399380,1.112051,0.583495,1.852890,0.772977,7.342087,0.416334,2.523592,2.016111,0.395846,1.777186,1.790616,1.122470
Topic1,2.708130,4.752740,1.939614,23.105211,15.676860,3.776225,0.564726,0.383152,0.341531,0.357302,2.636247,1.680876,7.882636,0.353943,2.364178,0.381125,0.370629,0.343111,0.669465,7.806409,0.447476,10.132652,0.561950,5.190343,8.424383,0.434858,6.712704,10.703251,25.120611,17.968149,13.125574,17.676152,0.679129,8.987139,14.358917,0.499003,2.404334,8.614618,4.589480,12.567986,0.412042,5.481190,0.392883,3.547256,27.428696,0.342647,148.253971,6.324470,7.271902,0.393276,1.478282,30.302465,0.844044,2.573160,8.083044,0.534739,10.294113,1.091794,2.703866,4.057833,0.379334,3.116186,0.331069,10.754131,33.682705,6.318407,6.963493,0.364273,0.493202,0.368111,4.320705,9.802604,0.513709,0.653733,13.525875,5.960713,8.855135,11.495863,0.352612,14.883937,4.777638,0.880455,8.122647,3.797387,0.636984,0.529211,4.953053,2.017280,0.552849,0.773420,5.572187,8.078269,0.598509,5.365788,0.931312,5.590739,0.364068,1.843992,3.485507,1.565882,5.219638,6.720023,0.392652,0.517059,1

### Get the 15 keywords each topic:


In [15]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=100):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=100)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords.head(10)

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19,Word 20,Word 21,Word 22,Word 23,Word 24,Word 25,Word 26,Word 27,Word 28,Word 29,Word 30,Word 31,Word 32,Word 33,Word 34,Word 35,Word 36,Word 37,Word 38,Word 39,Word 40,Word 41,Word 42,Word 43,Word 44,Word 45,Word 46,Word 47,Word 48,Word 49,Word 50,Word 51,Word 52,Word 53,Word 54,Word 55,Word 56,Word 57,Word 58,Word 59,Word 60,Word 61,Word 62,Word 63,Word 64,Word 65,Word 66,Word 67,Word 68,Word 69,Word 70,Word 71,Word 72,Word 73,Word 74,Word 75,Word 76,Word 77,Word 78,Word 79,Word 80,Word 81,Word 82,Word 83,Word 84,Word 85,Word 86,Word 87,Word 88,Word 89,Word 90,Word 91,Word 92,Word 93,Word 94,Word 95,Word 96,Word 97,Word 98,Word 99
Topic 0,duterte,say,oust,kayo,people,ask,face,think,lawyere,class,test,pilipina,start,supporter,doh,apologist,law,pre,naman,testing,address,cross,make,result,case,mayor,govt,read,bansa,lambaste,post,job,want,statement,come,president,covid,eleksyon,thing,number,hope,news,stop,investigation,night,attack,bata,report,admin,aquino,respond,issue,allow,fund,friend,student,use,niyo,lose,fact,philippine,remember,happen,fight,watch,dictator,regime,need,state,police,family,help,supply,group,way,oustduterte,talk,policy,amp,bbm,wag,world,work,look,story,order,return,tell,change,audit,press,probe,year,government,vote,politic,live,leave,hear,vice
Topic 1,duterte,government,election,drug,people,corruption,administration,run,president,country,support,covid,try,year,allege,kill,davao,time,war,interview,vaccine,pre,probe,court,defend,stop,work,watch,job,fund,continue,nation,regime,result,fact,campaign,help,term,dami,issue,office,press,deal,change,senator,land,experience,live,report,audit,bring,story,duterterapist,talk,group,mind,come,dutertemanyak,govt,want,investigation,order,make,life,response,supply,protect,dictatorship,state,look,policy,member,case,remember,leave,police,know,address,oust,death,supporter,way,hear,philippine,twitter,read,filipino,night,law,allow,doh,mean,post,use,testing,today,victim,freedom,pay,add
Topic 2,vote,president,make,marcose,history,need,tell,time,let,leader,year,candidate,opposition,know,want,power,talaga,truth,country,mean,believe,come,world,lol,people,puro,lie,record,neveragain,politician,family,thank,read,niyo,hear,amp,pala,response,change,presidency,oustduterte,aquino,register,freedom,law,add,life,vice,death,case,mind,respond,help,voter,nation,number,use,health,fight,regime,lose,support,thing,government,doh,work,twitter,win,filipino,attack,bansa,eleksyon,way,bring,run,hope,policy,duterte,watch,administration,supporter,dutertewakasan,think,happen,leni,medium,neverforget,philippine,address,dilawan,dami,allow,elcac,end,pre,start,supply,kill,fact,land
Topic 3,oustduterte,neveragain,law,people,amp,day,oustdutertenow,talk,today,duqueresign,dutertewakasan,way,elcac,let,nating,student,news,fight,health,come,happen,admin,anniversary,medium,pay,look,believe,member,say,number,night,bata,duterterapist,dutertemanyak,thing,presidency,power,thank,want,pilipina,know,voter,victim,cross,war,drug,story,register,mind,use,regime,state,change,protect,friend,leave,face,lose,experience,apologist,test,issue,tyranny,life,neverforget,kayo,bayan,government,senator,remember,hope,read,filipino,work,duterte,statement,leader,police,probe,defend,hahahaha,vote,make,policy,need,candidate,stop,covid,politician,deal,think,supporter,twitter,dictator,win,oust,end,press,administration,court
Topic 4,dilawan,neverforget,neveragain,bbm,law,country,aquino,daw,presidency,forget,bbmforpresident,end,bayan,hope,budget,hahahaha,politic,dictatorship,wake,look,leni,remember,win,wag,return,dictator,philippine,make,statement,order,tyranny,happen,friend,power,people,allow,victim,anniversary,regime,group,vice,medium,report,oustduterte,pilipina,duterte,leave,president,thing,life,pala,fact,pay,supply,night,protect,nation,say,administration,mayor,filipino,dutertewakasan,start,world,post,twitte

### Next, put the 5 topics we infered into the dataframe.


In [16]:
Topics = ["Political stance","Government Policies", "Election", "Filipino Political News","President"]
df_topic_keywords["Topics"]=Topics
df_topic_keywords.head()

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19,Word 20,Word 21,Word 22,Word 23,Word 24,Word 25,Word 26,Word 27,Word 28,Word 29,Word 30,Word 31,Word 32,Word 33,Word 34,Word 35,Word 36,Word 37,Word 38,Word 39,Word 40,Word 41,Word 42,Word 43,Word 44,Word 45,Word 46,Word 47,Word 48,Word 49,Word 50,Word 51,Word 52,Word 53,Word 54,Word 55,Word 56,Word 57,Word 58,Word 59,Word 60,Word 61,Word 62,Word 63,Word 64,Word 65,Word 66,Word 67,Word 68,Word 69,Word 70,Word 71,Word 72,Word 73,Word 74,Word 75,Word 76,Word 77,Word 78,Word 79,Word 80,Word 81,Word 82,Word 83,Word 84,Word 85,Word 86,Word 87,Word 88,Word 89,Word 90,Word 91,Word 92,Word 93,Word 94,Word 95,Word 96,Word 97,Word 98,Word 99,Topics
Topic 0,duterte,say,oust,kayo,people,ask,face,think,lawyere,class,test,pilipina,start,supporter,doh,apologist,law,pre,naman,testing,address,cross,make,result,case,mayor,govt,read,bansa,lambaste,post,job,want,statement,come,president,covid,eleksyon,thing,number,hope,news,stop,investigation,night,attack,bata,report,admin,aquino,respond,issue,allow,fund,friend,student,use,niyo,lose,fact,philippine,remember,happen,fight,watch,dictator,regime,need,state,police,family,help,supply,group,way,oustduterte,talk,policy,amp,bbm,wag,world,work,look,story,order,return,tell,change,audit,press,probe,year,government,vote,politic,live,leave,hear,vice,Political stance
Topic 1,duterte,government,election,drug,people,corruption,administration,run,president,country,support,covid,try,year,allege,kill,davao,time,war,interview,vaccine,pre,probe,court,defend,stop,work,watch,job,fund,continue,nation,regime,result,fact,campaign,help,term,dami,issue,office,press,deal,change,senator,land,experience,live,report,audit,bring,story,duterterapist,talk,group,mind,come,dutertemanyak,govt,want,investigation,order,make,life,response,supply,protect,dictatorship,state,look,policy,member,case,remember,leave,police,know,address,oust,death,supporter,way,hear,philippine,twitter,read,filipino,night,law,allow,doh,mean,post,use,testing,today,victim,freedom,pay,add,Government Policies
Topic 2,vote,president,make,marcose,history,need,tell,time,let,leader,year,candidate,opposition,know,want,power,talaga,truth,country,mean,believe,come,world,lol,people,puro,lie,record,neveragain,politician,family,thank,read,niyo,hear,amp,pala,response,change,presidency,oustduterte,aquino,register,freedom,law,add,life,vice,death,case,mind,respond,help,voter,nation,number,use,health,fight,regime,lose,support,thing,government,doh,work,twitter,win,filipino,attack,bansa,eleksyon,way,bring,run,hope,policy,duterte,watch,administration,supporter,dutertewakasan,think,happen,leni,medium,neverforget,philippine,address,dilawan,dami,allow,elcac,end,pre,start,supply,kill,fact,land,Election
Topic 3,oustduterte,neveragain,law,people,amp,day,oustdutertenow,talk,today,duqueresign,dutertewakasan,way,elcac,let,nating,student,news,fight,health,come,happen,admin,anniversary,medium,pay,look,believe,member,say,number,night,bata,duterterapist,dutertemanyak,thing,presidency,power,thank,want,pilipina,know,voter,victim,cross,war,drug,story,register,mind,use,regime,state,change,protect,friend,leave,face,lose,experience,apologist,test,issue,tyranny,life,neverforget,kayo,bayan,government,senator,remember,hope,read,filipino,work,duterte,statement,leader,police,probe,defend,hahahaha,vote,make,policy,need,candidate,stop,covid,politician,deal,think,supporter,twitter,dictator,win,oust,end,press,administration,court,Filipino Political News
Topic 4,dilawan,neverforget,neveragain,bbm,law,country,aquino,daw,presidency,forget,bbmforpresident,end,bayan,hope,budget,hahahaha,politic,dictatorship,wake,look,leni,remember,win,wag,return,dictator,philippine,make,statement,order,tyranny,happen,friend,power,people,allow,victim,anniversary,regime,group,vice,medium,report,oustduterte,pilipina,duterte,leave,president,thing,life,pala,fact,pay,supply,night,protect,na